In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import getpass
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

/home/kgvt/project_agent/agent_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def _set_env():
    load_dotenv()
    if "GOOGLE_API_KEY" not in os.environ:
        os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")


_set_env()

In [3]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [4]:
# https://www.moneycontrol.com/

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [os.environ["Stocks"]]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=100, chunk_overlap=20
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings,
)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
results = vectorstore.similarity_search(
    "idfc bank shares up",
    k=2,
    # filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* HDFC Bank

1742.60
116.45
134.88



ITC

482.10
-27.80
82.16



Larsen

3327.80
-353.35
77.61




See all Most Active









Top Gainers


Nifty
Sensex








Company
Price
Change [{'description': 'Business News Today: Read the latest business news on the Indian economy, global market, upcoming IPOs and more. Get Live Stock Price, Stock and Share market news, Finance News, Sensex, Nifty Live, Commodity Market, IPO news, economy news, and personal finance news today only at Moneycontrol', 'language': 'en', 'source': 'https://www.moneycontrol.com/', 'title': 'Business News Today: Stock and Share Market News, Economy and Finance News, Sensex, Nifty, Global Market, NSE, BSE Live IPO News'}]
* IDFC First Bank Q2 Results: Net profit down 73% at Rs 200.7 cr; NII up 21% [{'description': 'Business News Today: Read the latest business news on the Indian economy, global market, upcoming IPOs and more. Get Live Stock Price, Stock and Share market news, Finance News, Sensex, Nifty Live, Commod

In [6]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "retriver_stock_market_updates",
    "Search and return information about stocks in news whether they are good news or bad news , , bullish and bearnish newses as well",
)

tools = [retriever_tool]